In [1]:
! pip install path

  Obtaining dependency information for path from https://files.pythonhosted.org/packages/55/49/5ffc63f96029e4f899d6caee9db40c445ded1bf15cb60b32d77363c2c8de/path-17.0.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
! pip install catboost


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
! pip install torch torchvision torchaudio --index-url /usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py

Looking in indexes: /usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from path import Path

In [3]:
data_path=Path('../data')
data_path.glob('*.csv')

[]

In [7]:
train = pd.read_csv('train.csv')

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

ModuleNotFoundError: No module named 'torch'

In [8]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['речь']
        self.targets = dataframe['дефолт']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index][1:-1])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

NameError: name 'RobertaTokenizer' is not defined

In [29]:
data = pd.read_csv('train.csv')
from sklearn.model_selection import train_test_split
train, val = train_test_split(data,test_size=0.5,random_state=52)
val, test = train_test_split(val,test_size=0.2,random_state=52)
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
# трейн 0,5 вал 0.4 тест 0,1

In [30]:
train_params = {'batch_size': 64,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 64,
                'shuffle': False,
                'num_workers': 0
                }

training_set = SentimentData(train, tokenizer, 256)
testing_set = SentimentData(val, tokenizer, 256)

training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(testing_set, **test_params)

In [31]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.embedder = torch.nn.Linear(768, 32)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(32, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)

        emb = self.embedder(pooler)
        pooler = torch.nn.ReLU()(emb)
        output = self.classifier(pooler)
        return output, emb

In [1]:
device=torch.device('cuda')

NameError: name 'torch' is not defined

In [ ]:
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [33]:
!nvidia-smi

Sat Dec 28 15:17:40 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8              3W /   50W |     145MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [34]:
model = RobertaClass()
model.to(device);

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: Torch not compiled with CUDA enabled

In [19]:
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-06)

In [21]:
from sklearn.metrics import f1_score, roc_auc_score
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [23]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model
from sklearn.metrics import f1_score, roc_auc_score

def train_model(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    targets_all=[]
    scores_all=[]
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs, emb = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        targets_all.append(targets.cpu().numpy())
        scores_all.append(outputs.softmax(dim=1)[..., 1].detach().cpu().numpy())

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)

        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples


            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")


        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    roc_auc=roc_auc_score(np.hstack(targets_all),
                          np.hstack(scores_all)
                         )
    targets_all=[]
    scores_all=[]

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}.")
    print(f"Training AUC Epoch: {roc_auc}.")
    return roc_auc

def valid_model(model, testing_loader, roc_auc_train):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0

    targets_all=[]
    scores_all=[]

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.long)
            outputs, emb = model(ids, mask, token_type_ids)
            outputs=outputs.squeeze()
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(outputs.data, dim=1)
            n_correct += calcuate_accuracy(big_idx, targets)

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)

            targets_all.append(targets.cpu().numpy())
            scores_all.append(outputs.softmax(dim=1)[..., 1].detach().cpu().numpy())
            # if _%5000==0:
            #     loss_step = tr_loss/nb_tr_steps
            #     accu_step = (n_correct*100)/nb_tr_examples
            #     print(f"Validation Loss per 100 steps: {loss_step}")
            #     print(f"Validation Accuracy per 100 steps: {accu_step}")

    roc_auc=roc_auc_score(np.hstack(targets_all),
                          np.hstack(scores_all)
                         )
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    print(f"Validation auc Epoch: {roc_auc}")

    output_model_file = f'roberts_x_emb_{roc_auc_train}_{roc_auc}.bin'
    torch.save(model, output_model_file)


    return


In [27]:
EPOCHS = 5
for epoch in range(EPOCHS):
    ra=train_model(epoch)
    valid_model(model, val_loader, ra )

0it [00:00, ?it/s]


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
bert=torch.load('./roberts_x_emb_0.9665052141094749_0.9764164500312258.bin')

<ipython-input-31-77afdcc5f195>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bert=torch.load('./roberts_x_emb_0.9665052141094749_0.9764164500312258.bin')


OutOfMemoryError: CUDA out of memory. Tried to allocate 148.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 3.06 MiB is free. Process 7983 has 14.74 GiB memory in use. Of the allocated memory 14.54 GiB is allocated by PyTorch, and 71.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# data = pd.read_csv(data_path/'train.csv')
# from sklearn.model_selection import train_test_split
# train, val = train_test_split(data,test_size=0.15,random_state=52)
# train.reset_index(drop=True, inplace=True)
# val.reset_index(drop=True, inplace=True)

In [ ]:
# bert=RobertaModel.from_pretrained("roberta-base").to(device)

In [ ]:
training_set = SentimentData(train, tokenizer, 256)
val_set = SentimentData(val, tokenizer, 256)
test_set = SentimentData(test, tokenizer, 256)
training_loader = DataLoader(training_set, batch_size=64, shuffle=False)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [ ]:
train_embs=[]
for batch in tqdm(training_loader):
    ids = batch['ids'].to(device, dtype = torch.long)
    mask = batch['mask'].to(device, dtype = torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    outputs, emb = bert(ids, mask, token_type_ids)
    outputs=outputs.softmax(dim=1)[:, 1].detach().cpu().unsqueeze(dim=-1).numpy()
    emb=emb.detach().cpu().numpy()
    emb=np.concatenate([emb, outputs], axis=1)
    train_embs.append(emb)

val_embs=[]
for batch in tqdm(val_loader):
    ids = batch['ids'].to(device, dtype = torch.long)
    mask = batch['mask'].to(device, dtype = torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    outputs, emb = bert(ids, mask, token_type_ids)
    outputs=outputs.softmax(dim=1)[:, 1].detach().cpu().unsqueeze(dim=-1).numpy()
    emb=emb.detach().cpu().numpy()
    emb=np.concatenate([emb, outputs], axis=1)
    val_embs.append(emb)

test_embs=[]
for batch in tqdm(test_loader):
    ids = batch['ids'].to(device, dtype = torch.long)
    mask = batch['mask'].to(device, dtype = torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    outputs, emb = bert(ids, mask, token_type_ids)
    outputs=outputs.softmax(dim=1)[:, 1].detach().cpu().unsqueeze(dim=-1).numpy()
    emb=emb.detach().cpu().numpy()
    emb=np.concatenate([emb, outputs], axis=1)
    test_embs.append(emb)

# train_embs=[]
# for batch in tqdm(training_loader):
#     ids = batch['ids'].to(device, dtype = torch.long)
#     mask = batch['mask'].to(device, dtype = torch.long)
#     token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
#     emb = bert(ids, mask, token_type_ids)[0][:, 0]
#     emb=emb.detach().cpu().numpy()
#     train_embs.append(emb)

# val_embs=[]
# for batch in tqdm(val_loader):
#     ids = batch['ids'].to(device, dtype = torch.long)
#     mask = batch['mask'].to(device, dtype = torch.long)
#     token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
#     emb = bert(ids, mask, token_type_ids)[0][:, 0]
#     emb=emb.detach().cpu().numpy()
#     val_embs.append(emb)

# test_embs=[]
# for batch in tqdm(test_loader):
#     ids = batch['ids'].to(device, dtype = torch.long)
#     mask = batch['mask'].to(device, dtype = torch.long)
#     token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
#     emb = bert(ids, mask, token_type_ids)[0][:, 0]
#     emb=emb.detach().cpu().numpy()
#     test_embs.append(emb)

  0%|          | 0/63 [00:00<?, ?it/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
train_embs=np.concatenate(train_embs, axis=0)
val_embs=np.concatenate(val_embs, axis=0)
test_embs=np.concatenate(test_embs, axis=0)

ValueError: need at least one array to concatenate

In [ ]:
# test_embs.shape

In [ ]:
cat_features=['лимит_нарушен',
 'пол',
 'тип',
 'цель',
 'кредитоспособность',
 'другие_кредиты',
 'бизнес',
 'амортизация',
 'только_процент',
 'один_платеж',
 'работа',
 'тип_залога',
 'тип_кредита',
 'возраст',
 'прямой_залог',
 'сбор']

In [ ]:
X_train, X_val, X_test = train.drop(['дефолт', 'ID'], axis=1), val.drop(['дефолт', 'ID'], axis=1),  test.drop(['дефолт', 'ID'], axis=1)
y_train, y_val, y_test,  = train['дефолт'], val['дефолт'], test['дефолт']

In [ ]:
for i in range(test_embs.shape[1]):
    X_train[f'emb_{i}']=train_embs[:, i]
    X_val[f'emb_{i}']=val_embs[:, i]
    X_test[f'emb_{i}']=test_embs[:, i]
# X_train[f'emb_{32}']=train_embs[:, 32]
# X_test[f'emb_{32}']=val_embs[:, 32]

NameError: name 'test_embs' is not defined

In [ ]:
for cat in cat_features:
    print(cat, X_train[cat].unique())

    if X_train[cat].dtype==float or X_train[cat].dtype==int:
        X_train[cat]=X_train[cat].fillna(-1.)
        X_val[cat]=X_val[cat].fillna(-1.)
        X_test[cat]=X_test[cat].fillna(-1.)

        X_train[cat]=X_train[cat].astype(int)
        X_val[cat]=X_val[cat].astype(int)
        X_test[cat]=X_test[cat].astype(int)
    else:
        X_train[cat]=X_train[cat].fillna('')
        X_val[cat]=X_val[cat].fillna('')
        X_test[cat]=X_test[cat].fillna('')


лимит_нарушен [ 0. nan  1.]
пол ['м' nan 'ж']
тип [ 3. nan  2.  1.]
цель [ 0. nan  1.  3.  2.]
кредитоспособность [ 1. nan  0.]
другие_кредиты [nan  0.  1.]
бизнес [nan  0.  1.]
амортизация [ 0. nan  1.]
только_процент [ 0.  1. nan]
один_платеж [nan  0.  1.]
работа ['осн' 'инвест' 'втор']
тип_залога ['дом' 'земля']
тип_кредита [ 2.  1. nan  4.  3.]
возраст ['45-54' '35-44' '25-34' '65-74' '55-64' '>74' '<25' nan]
прямой_залог [ 1. nan  0.]
сбор ['middle' 'not Lilliputian' 'popular' 'average' 'whopping' 'requisite'
 'Zero' "Whole lotta nothin'" 'deficiency' 'modest' 'passable' 'essential'
 'not sparse' 'Not a soul' 'Not a thing' 'Nothingness' 'not substantial'
 'extravagant' 'Blank slate' 'normal' 'not largish' 'Not a trace'
 'colossal' 'not pocket-sized' 'exorbitant' 'need' 'little' 'minus'
 'midsize' 'moderate' 'tolerable' 'Zippo' 'not boxcar' 'staggering'
 'not wanting' 'regular' 'not microscopic' 'requirement' 'famishment'
 'not bantam' 'necessity' 'small' 'reasonable' 'spacious' 'u

In [ ]:
y_val

,дефолт
0,0
1,0
2,0
3,0
4,0
...,...
3195,0
3196,0
3197,0
3198,0


In [ ]:
from catboost import Pool
from catboost import CatBoostClassifier

train_pool = Pool(
    X_val,
    y_val,
    cat_features=cat_features,
    text_features=['речь'],
    # feature_names=list(X_train)
)
valid_pool = Pool(
    X_test,
    y_test,
    cat_features=cat_features,
    text_features=['речь'],
    # feature_names=list(X_train)
)

catboost_params = {
    'iterations': 3000,
    'learning_rate': 0.01,
    'eval_metric': 'F1',
    'task_type': 'CPU',
    'verbose': 50,
}
# clas_weight = [5,4]
model = CatBoostClassifier(**catboost_params,
                           use_best_model=True,
                           l2_leaf_reg=2,
                           early_stopping_rounds=300,
                           tokenizers=[
                                {
                                    'tokenizer_id': 'Sense',
                                    'separator_type': 'BySense',
                                    'lowercasing': 'True',
                                    'token_types':['Word', 'Number', 'SentenceBreak'],
                                    'sub_tokens_policy':'SeveralTokens'
                                }
                            ],
                            dictionaries = [
                                {
                                    'dictionary_id': 'word',
                                    'max_dictionary_size': '50000'
                                }
                            ],
                            feature_calcers = [
                                'BM25',
                                'BoW:top_tokens_count=10000',
                                'NaiveBayes',
                            ]
                           # class_weights=class_weight
                          )
model.fit(train_pool, eval_set=valid_pool)

0:	learn: 0.0078844	test: 0.0109890	best: 0.0109890 (0)	total: 199ms	remaining: 9m 56s
50:	learn: 0.7923698	test: 0.8121212	best: 0.8121212 (50)	total: 4.44s	remaining: 4m 16s
100:	learn: 0.8919308	test: 0.9117647	best: 0.9117647 (99)	total: 10.4s	remaining: 4m 59s
150:	learn: 0.9209237	test: 0.9212828	best: 0.9244186 (139)	total: 16.8s	remaining: 5m 16s
200:	learn: 0.9381018	test: 0.9306358	best: 0.9306358 (178)	total: 24.3s	remaining: 5m 38s
250:	learn: 0.9438356	test: 0.9310345	best: 0.9310345 (232)	total: 28.5s	remaining: 5m 12s
300:	learn: 0.9517335	test: 0.9314286	best: 0.9314286 (293)	total: 33.6s	remaining: 5m 1s
350:	learn: 0.9566982	test: 0.9375000	best: 0.9375000 (343)	total: 39.5s	remaining: 4m 58s
400:	learn: 0.9609164	test: 0.9435028	best: 0.9435028 (385)	total: 44s	remaining: 4m 44s
450:	learn: 0.9657948	test: 0.9435028	best: 0.9435028 (385)	total: 48.7s	remaining: 4m 35s
500:	learn: 0.9692513	test: 0.9494382	best: 0.9494382 (460)	total: 54.8s	remaining: 4m 33s
550:	lear

In [ ]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,сбор,46.748252
1,речь,38.929959
2,тип_кредита,12.436833
3,стоимость_имущества,0.308557
4,амортизация,0.307119
5,только_процент,0.150094
6,работа,0.139252
7,цель,0.125595
8,тип,0.124485
9,один_платеж,0.120098


In [ ]:
train_proba=model.predict_proba(X_val)[:, 1]

In [ ]:
from sklearn.metrics import f1_score
best_trsh=0
max_f1=0
for trsh in np.linspace(0, 1.,100 ):
    f1=f1_score(y_val, (train_proba>trsh).astype(int))
    if f1>max_f1:
        best_trsh=trsh
        max_f1=f1

f1_score(y_val, (train_proba>best_trsh).astype(int)), best_trsh

(0.9814814814814815, 0.43434343434343436)

In [ ]:
test_proba=model.predict_proba(X_test)[:, 1]
from sklearn.metrics import f1_score
best_trsh=0
max_f1=0
for trsh in np.linspace(0, 1.,100 ):
    f1=f1_score(y_test, (test_proba>trsh).astype(int))
    if f1>max_f1:
        best_trsh=trsh
        max_f1=f1

f1_score(y_test, (test_proba>best_trsh).astype(int)), best_trsh

(0.9726775956284153, 0.32323232323232326)

In [ ]:
subm=pd.read_csv('/content/test_feat.csv')
expl_subm=pd.read_csv('/content/submission_example.csv')
submission=pd.DataFrame(subm['ID'])
submission.head(2)
# subm=subm[X_train.columns]

,ID
0,d61968aa-7fb8-46b2-8573-3d101d3ebe5b
1,760116cf-25b4-47aa-9a20-e3baa5fc6d09


In [ ]:
subm['дефолт']=0
subm_set = SentimentData(subm, tokenizer, 256)
subm_loader = DataLoader(subm_set, batch_size=64, shuffle=False)

In [ ]:
subm_embs=[]
for batch in tqdm(subm_loader):
    ids = batch['ids'].to(device, dtype = torch.long)
    mask = batch['mask'].to(device, dtype = torch.long)
    token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
    outputs, emb = bert(ids, mask, token_type_ids)
    outputs=outputs.softmax(dim=1)[:, 1].detach().cpu().unsqueeze(dim=-1).numpy()
    emb=emb.detach().cpu().numpy()
    emb=np.concatenate([emb, outputs], axis=1)
    subm_embs.append(emb)

  0%|          | 0/32 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2673: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  0%|          | 0/32 [00:00<?, ?it/s]


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
subm_embs=np.concatenate(subm_embs, axis=0)
for i in range(33):
    subm[f'emb_{i}']=subm_embs[:, i]


ValueError: need at least one array to concatenate

In [ ]:
subm=subm[X_train.columns]

In [ ]:
for cat in cat_features:
    # print(cat, subm[cat].unique())

    if subm[cat].dtype==float or subm[cat].dtype==int:
        subm[cat]=subm[cat].fillna(-1.)

        subm[cat]=subm[cat].astype(int)

    else:
        subm[cat]=subm[cat].fillna('')


<ipython-input-56-1b7c99287e02>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subm[cat]=subm[cat].fillna(-1.)
<ipython-input-56-1b7c99287e02>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subm[cat]=subm[cat].astype(int)
<ipython-input-56-1b7c99287e02>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [ ]:
y_pred=model.predict_proba(subm)[:, 1]
y_pred=(y_pred>0.45).astype(int)
y_pred.sum()

460

In [ ]:
submission['дефолт']=y_pred
submission.to_csv('sub.csv', index=False)

In [ ]:
y_pred.sum()

460